In [239]:
import os
import numpy as np
from error_parity import RelaxedThresholdOptimizer
from error_parity.classifiers import RandomizedClassifier

import torch
import torch.nn as nn

from source.constants import RESULTS_PATH, PLOTS_PATH
from source.data.face_detection import get_fair_face, get_utk
from source.utils.metrics import accuracy, aod, eod, spd

os.makedirs(PLOTS_PATH, exist_ok=True)

In [240]:
method_seeds = [42, 142, 242, 342, 442]
dseed = 42

model = ["resnet18", "resnet34", "resnet50"][2]

verbose = False

targets = ["age", "gender", "race(old)", "race"]
# predicting race does not give high unfairness (with either pa) for eod and aod
# predicting gender also not too nice (only unfairness with age)
target = 3 # 0, 1, 2, 3
pa = 1 # 0, 1, 2, 3

In [241]:
# parameters
c = 2
constraint = ["demographic_parity", "true_positive_rate_parity", "average_odds"][c]

In [242]:
# no need to define targets and protected attributes, are queried directly afterwards
ff_train_ds, ff_test_ds = get_fair_face(binarize=True, augment=False)
utk_test_ds = get_utk(binarize=True)

run_path = os.path.join(RESULTS_PATH, f"fairface_target{target}_{model}_mseed{method_seeds[0]}_dseed{dseed}")
fair_inds = torch.load(os.path.join(run_path, "fair_inds.pt"))
val_inds = torch.load(os.path.join(run_path, "val_inds.pt"))

print(len(fair_inds), len(val_inds), len(ff_test_ds), len(utk_test_ds))

# get targets and protected attributes
y_fair_t = ff_train_ds.targets[target, fair_inds]
a_fair_t = ff_train_ds.targets[pa, fair_inds]
y_val_t = ff_train_ds.targets[target, val_inds]
a_val_t = ff_train_ds.targets[pa, val_inds]
y_ff_test_t = ff_test_ds.targets[target]
a_ff_test_t = ff_test_ds.targets[pa]
y_utk_test_t = utk_test_ds.targets[target]
a_utk_test_t = utk_test_ds.targets[pa]

10843 10843 10954 23705


In [243]:
# load probits
fair_probits, val_probits, ff_test_probits, utk_test_probits = list(), list(), list(), list()
for mseed in method_seeds:
    path = os.path.join(RESULTS_PATH, f"fairface_target{target}_{model}_mseed{mseed}_dseed{dseed}")

    fair_probits.append(torch.load(os.path.join(path, f"fair_probits_t{target}.pt")))
    val_probits.append(torch.load(os.path.join(path, f"val_probits_t{target}.pt")))
    ff_test_probits.append(torch.load(os.path.join(path, f"ff_test_probits_t{target}.pt")))
    utk_test_probits.append(torch.load(os.path.join(path, f"utk_test_probits_t{target}.pt")))

In [244]:
# calculate accuracies and fairness measures
val_accs, ff_test_accs, utk_test_accs = list(), list(), list()
val_spds, ff_test_spds, utk_test_spds = list(), list(), list()
val_eods, ff_test_eods, utk_test_eods = list(), list(), list()
val_aods, ff_test_aods, utk_test_aods = list(), list(), list()

for m in range(len(method_seeds)):
    val_accs.append([accuracy(p.argmax(dim=1), y_val_t) for p in val_probits[m]])
    ff_test_accs.append([accuracy(p.argmax(dim=1), y_ff_test_t) for p in ff_test_probits[m]])
    utk_test_accs.append([accuracy(p.argmax(dim=1), y_utk_test_t) for p in utk_test_probits[m]])

    val_spds.append([spd(p.argmax(dim=1), a_val_t) for p in val_probits[m]])
    ff_test_spds.append([spd(p.argmax(dim=1), a_ff_test_t) for p in ff_test_probits[m]])
    utk_test_spds.append([spd(p.argmax(dim=1), a_utk_test_t) for p in utk_test_probits[m]])
    
    val_eods.append([eod(p.argmax(dim=1), y_val_t, a_val_t) for p in val_probits[m]])
    ff_test_eods.append([eod(p.argmax(dim=1), y_ff_test_t, a_ff_test_t) for p in ff_test_probits[m]])
    utk_test_eods.append([eod(p.argmax(dim=1), y_utk_test_t, a_utk_test_t) for p in utk_test_probits[m]])

    val_aods.append([aod(p.argmax(dim=1), y_val_t, a_val_t) for p in val_probits[m]])
    ff_test_aods.append([aod(p.argmax(dim=1), y_ff_test_t, a_ff_test_t) for p in ff_test_probits[m]])
    utk_test_aods.append([aod(p.argmax(dim=1), y_utk_test_t, a_utk_test_t) for p in utk_test_probits[m]])
    

In [245]:
# method to do the fake predictions
class DummyPredictor(nn.Module):
    def __init__(self, probits):
        super(DummyPredictor, self).__init__()
        self.probits = probits

    def forward(self, indices:torch.Tensor):
        return self.probits[indices].numpy()

In [246]:
def get_thresholds(fair_clf, verbose=True):
    thresholds = list()
    for i in range(2):
        if verbose: print(f"Class {i}")
        if isinstance(fair_clf._realized_classifier.group_to_clf[i], RandomizedClassifier):
            thrs = list()
            for clf in fair_clf._realized_classifier.group_to_clf[i].classifiers:
                if verbose: print(clf.threshold)
                thrs.append(clf.threshold)
            thresholds.append(thrs)   
        else:
            thrs = fair_clf._realized_classifier.group_to_clf[i].threshold
            if verbose: print(thrs)
            thresholds.append([thrs, thrs])
    return thresholds

## FairFace

Optimize Ensemble for average member constraint

In [247]:
accs_bma, fairs_bma = list(), list()
accs_bma_pp, fairs_bma_pp = list(), list()
accs_avg, fairs_avg = list(), list()
thresholds_bma_pp = list()

for m in range(len(method_seeds)):

    if verbose: print("-"*20 + f"  seed {m}  " + "-"*20)

    val_m_probits = torch.mean(val_probits[m], dim=0)

    val_fairness = [val_spds[m], val_eods[m], val_aods[m]][c]
    test_fairness = [ff_test_spds[m], ff_test_eods[m], ff_test_aods[m]][c]

    model = DummyPredictor(val_m_probits)

    # Given any trained model that outputs real-valued scores
    fair_clf = RelaxedThresholdOptimizer(
        predictor=lambda X: model(X)[:, -1],   # for sklearn API
        constraint=constraint,
        tolerance=max(np.mean(val_fairness), 0), # fairness constraint tolerance
    )

    # Fit the fairness adjustment on some data
    # This will find the optimal _fair classifier_
    fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

    # Get the thresholds for the optimal classifier
    thresholds_bma_pp.append(get_thresholds(fair_clf, verbose=verbose))

    # overwrite model for predictor
    ff_test_m_probits = torch.mean(ff_test_probits[m], dim=0)
    model.probits = ff_test_m_probits

    # Now you can use `fair_clf` as any other classifier
    # You have to provide group information to compute fair predictions
    y_pred_test = fair_clf(X=torch.tensor(range(len(y_ff_test_t))), group=a_ff_test_t.numpy())
    y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

    if verbose: print("Avg Member")
    accs_avg.extend(ff_test_accs[m])
    if verbose: print(f"  {(ff_test_accs[0][m]):.3f}")
    fairs_avg.extend(test_fairness)
    if verbose: print(f"  {test_fairness[0]:.3f} (val: {val_fairness[0]:.3f})")
    if verbose: print("BMA")
    accs_bma.append(accuracy(ff_test_m_probits.argmax(dim=1), y_ff_test_t).item())
    if verbose: print(f"  {(accuracy(ff_test_m_probits.argmax(dim=1), y_ff_test_t).item()):.3f}")
    if c == 0:
        fairs_bma.append(spd(ff_test_m_probits.argmax(dim=1), a_ff_test_t).item())
        if verbose: print(f"  {spd(ff_test_m_probits.argmax(dim=1), a_ff_test_t).item():.3f}")
    elif c == 1:
        fairs_bma.append(eod(ff_test_m_probits.argmax(dim=1), y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {eod(ff_test_m_probits.argmax(dim=1), y_ff_test_t, a_ff_test_t).item():.3f}")
    elif c == 2:
        fairs_bma.append(aod(ff_test_m_probits.argmax(dim=1), y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {aod(ff_test_m_probits.argmax(dim=1), y_ff_test_t, a_ff_test_t).item():.3f}")
    if verbose: print("BMA-PP")
    accs_bma_pp.append(accuracy(y_pred_test, y_ff_test_t).item())
    if verbose: print(f"  {(accuracy(y_pred_test, y_ff_test_t).item()):.3f}")
    if c == 0:
        fairs_bma_pp.append(spd(y_pred_test, a_ff_test_t).item())
        if verbose: print(f"  {spd(y_pred_test, a_ff_test_t).item():.3f}")
    elif c == 1:
        fairs_bma_pp.append(eod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {eod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")
    elif c == 2:
        fairs_bma_pp.append(aod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {aod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")

thresholds_bma_pp = np.asarray(thresholds_bma_pp)

print("-"*30)
print(f"${np.mean(accs_avg):.3f}_{'{'}\pm {np.std(accs_avg):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_avg):.3f}_{'{'}\pm {np.std(fairs_avg):.3f}{'}'}$")
print(f"${np.mean(accs_bma):.3f}_{'{'}\pm {np.std(accs_bma):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma):.3f}_{'{'}\pm {np.std(fairs_bma):.3f}{'}'}$")
print(f"${np.mean(accs_bma_pp):.3f}_{'{'}\pm {np.std(accs_bma_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma_pp):.3f}_{'{'}\pm {np.std(fairs_bma_pp):.3f}{'}'}$")
print("-"*30)
for i in range(2):
    print(f"Group {i}")
    print(f"${np.mean(thresholds_bma_pp[:, i, 0]):.3f}_{'{'}\pm {np.std(thresholds_bma_pp[:, i, 0]):.3f}{'}'}$")
    print(f"${np.mean(thresholds_bma_pp[:, i, 1]):.3f}_{'{'}\pm {np.std(thresholds_bma_pp[:, i, 1]):.3f}{'}'}$")

------------------------------
$0.873_{\pm 0.002}$ & $0.013_{\pm 0.006}$
$0.888_{\pm 0.001}$ & $0.016_{\pm 0.002}$
$0.888_{\pm 0.001}$ & $0.016_{\pm 0.005}$
------------------------------
Group 0
$0.439_{\pm 0.049}$
$0.408_{\pm 0.044}$
Group 1
$0.345_{\pm 0.033}$
$0.337_{\pm 0.030}$


Optimize both ensemble and single model for 0.05 constraint

In [248]:
accs_bma_pp, fairs_bma_pp = list(), list()
accs_member_pp, fairs_member_pp = list(), list()
thresholds_bma_pp = list()
thresholds_member_pp = list()

for m in range(len(method_seeds)):

    if verbose: print("-"*20 + f"  seed {m}  " + "-"*20)

    val_m_probits = torch.mean(val_probits[m], dim=0)

    model = DummyPredictor(val_m_probits)

    # Given any trained model that outputs real-valued scores
    fair_clf = RelaxedThresholdOptimizer(
        predictor=lambda X: model(X)[:, -1],   # for sklearn API
        constraint=constraint,
        tolerance=0.05, # fairness constraint tolerance
    )

    # Fit the fairness adjustment on some data
    # This will find the optimal _fair classifier_
    fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

    # Get the thresholds for the optimal classifier
    thresholds_bma_pp.append(get_thresholds(fair_clf, verbose=verbose))

    # overwrite model for predictor
    ff_test_m_probits = torch.mean(ff_test_probits[m], dim=0)
    model.probits = ff_test_m_probits

    # Now you can use `fair_clf` as any other classifier
    # You have to provide group information to compute fair predictions
    y_pred_test = fair_clf(X=torch.tensor(range(len(y_ff_test_t))), group=a_ff_test_t.numpy())
    y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

    if verbose: print("BMA-PP")
    accs_bma_pp.append(accuracy(y_pred_test, y_ff_test_t).item())
    if verbose: print(f"  {(accuracy(y_pred_test, y_ff_test_t).item()):.3f}")
    if c == 0:
        fairs_bma_pp.append(spd(y_pred_test, a_ff_test_t).item())
        if verbose: print(f"  {spd(y_pred_test, a_ff_test_t).item():.3f}")
    elif c == 1:
        fairs_bma_pp.append(eod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {eod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")
    elif c == 2:
        fairs_bma_pp.append(aod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
        if verbose: print(f"  {aod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")

    for mem in range(len(val_probits[m])):
        val_m_probits = val_probits[m][mem]

        model = DummyPredictor(val_m_probits)

        # Given any trained model that outputs real-valued scores
        fair_clf = RelaxedThresholdOptimizer(
            predictor=lambda X: model(X)[:, -1],   # for sklearn API
            constraint=constraint,
            tolerance=0.05, # fairness constraint tolerance
        )

        # Fit the fairness adjustment on some data
        # This will find the optimal _fair classifier_
        fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

        # Get the thresholds for the optimal classifier
        thresholds_member_pp.append(get_thresholds(fair_clf, verbose=verbose))

        # overwrite model for predictor
        ff_test_m_probits = ff_test_probits[m][0]
        model.probits = ff_test_m_probits

        # Now you can use `fair_clf` as any other classifier
        # You have to provide group information to compute fair predictions
        y_pred_test = fair_clf(X=torch.tensor(range(len(y_ff_test_t))), group=a_ff_test_t.numpy())
        y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

        if mem == 0 and verbose : print("Member-PP")
        accs_member_pp.append(accuracy(y_pred_test, y_ff_test_t).item())
        if mem == 0 and verbose : print(f"  {(accuracy(y_pred_test, y_ff_test_t).item()):.3f}")
        if c == 0:
            fairs_member_pp.append(spd(y_pred_test, a_ff_test_t).item())
            if mem == 0 and verbose : print(f"  {spd(y_pred_test, a_ff_test_t).item():.3f}")
        elif c == 1:
            fairs_member_pp.append(eod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
            if mem == 0 and verbose : print(f"  {eod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")
        elif c == 2:
            fairs_member_pp.append(aod(y_pred_test, y_ff_test_t, a_ff_test_t).item())
            if mem == 0 and verbose : print(f"  {aod(y_pred_test, y_ff_test_t, a_ff_test_t).item():.3f}")

thresholds_bma_pp = np.asarray(thresholds_bma_pp)
thresholds_member_pp = np.asarray(thresholds_member_pp).reshape((-1, 2, 2))

print("-"*30)
print(f"${np.mean(accs_bma_pp):.3f}_{'{'}\pm {np.std(accs_bma_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma_pp):.3f}_{'{'}\pm {np.std(fairs_bma_pp):.3f}{'}'}$")
print(f"${np.mean(accs_member_pp):.3f}_{'{'}\pm {np.std(accs_member_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_member_pp):.3f}_{'{'}\pm {np.std(fairs_member_pp):.3f}{'}'}$")
print("-"*30)
for i in range(2):
    print(f"Group {i}")
    print(f"${np.mean(thresholds_bma_pp[:, i, 0]):.3f}_{'{'}\pm {np.std(thresholds_bma_pp[:, i, 0]):.3f}{'}'}$")
    print(f"${np.mean(thresholds_bma_pp[:, i, 1]):.3f}_{'{'}\pm {np.std(thresholds_bma_pp[:, i, 1]):.3f}{'}'}$")
print("-"*30)
for i in range(2):
    print(f"Group {i}")
    print(f"${np.mean(thresholds_member_pp[:, i, 0]):.3f}_{'{'}\pm {np.std(thresholds_member_pp[:, i, 0]):.3f}{'}'}$")
    print(f"${np.mean(thresholds_member_pp[:, i, 1]):.3f}_{'{'}\pm {np.std(thresholds_member_pp[:, i, 1]):.3f}{'}'}$")

------------------------------
$0.888_{\pm 0.002}$ & $0.019_{\pm 0.004}$
$0.873_{\pm 0.004}$ & $0.029_{\pm 0.027}$
------------------------------
Group 0
$0.439_{\pm 0.049}$
$0.408_{\pm 0.044}$
Group 1
$0.345_{\pm 0.033}$
$0.337_{\pm 0.030}$
------------------------------
Group 0
$0.646_{\pm 0.162}$
$0.577_{\pm 0.165}$
Group 1
$0.628_{\pm 0.156}$
$0.566_{\pm 0.174}$


## UTKFaces

Optimize Ensemble for average member constraint

In [249]:
accs_bma, fairs_bma = list(), list()
accs_bma_pp, fairs_bma_pp = list(), list()
accs_avg, fairs_avg = list(), list()

for m in range(len(method_seeds)):

    if verbose: print("-"*20 + f"  seed {m}  " + "-"*20)

    val_m_probits = torch.mean(val_probits[m], dim=0)

    val_fairness = [val_spds[m], val_eods[m], val_aods[m]][c]
    test_fairness = [utk_test_spds[m], utk_test_eods[m], utk_test_aods[m]][c]

    model = DummyPredictor(val_m_probits)

    # Given any trained model that outputs real-valued scores
    fair_clf = RelaxedThresholdOptimizer(
        predictor=lambda X: model(X)[:, -1],   # for sklearn API
        constraint=constraint,
        tolerance=max(np.mean(val_fairness), 0), # fairness constraint tolerance
    )

    # Fit the fairness adjustment on some data
    # This will find the optimal _fair classifier_
    fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

    # overwrite model for predictor
    utk_test_m_probits = torch.mean(utk_test_probits[m], dim=0)
    model.probits = utk_test_m_probits

    # Now you can use `fair_clf` as any other classifier
    # You have to provide group information to compute fair predictions
    y_pred_test = fair_clf(X=torch.tensor(range(len(y_utk_test_t))), group=a_utk_test_t.numpy())
    y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

    if verbose: print("Avg Member")
    accs_avg.extend(utk_test_accs[m])
    if verbose: print(f"  {(utk_test_accs[0][m]):.3f}")
    fairs_avg.extend(test_fairness)
    if verbose: print(f"  {test_fairness[0]:.3f} (val: {val_fairness[0]:.3f})")

    if verbose: print("BMA")
    accs_bma.append(accuracy(utk_test_m_probits.argmax(dim=1), y_utk_test_t).item())
    if verbose: print(f"  {(accuracy(utk_test_m_probits.argmax(dim=1), y_utk_test_t).item()):.3f}")
    if c == 0:
        fairs_bma.append(spd(utk_test_m_probits.argmax(dim=1), a_utk_test_t).item())
        if verbose: print(f"  {spd(utk_test_m_probits.argmax(dim=1), a_utk_test_t).item():.3f}")
    elif c == 1:
        fairs_bma.append(eod(utk_test_m_probits.argmax(dim=1), y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {eod(utk_test_m_probits.argmax(dim=1), y_utk_test_t, a_utk_test_t).item():.3f}")
    elif c == 2:
        fairs_bma.append(aod(utk_test_m_probits.argmax(dim=1), y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {aod(utk_test_m_probits.argmax(dim=1), y_utk_test_t, a_utk_test_t).item():.3f}")
    if verbose: print("BMA-PP")
    accs_bma_pp.append(accuracy(y_pred_test, y_utk_test_t).item())
    if verbose: print(f"  {(accuracy(y_pred_test, y_utk_test_t).item()):.3f}")
    if c == 0:
        fairs_bma_pp.append(spd(y_pred_test, a_utk_test_t).item())
        if verbose: print(f"  {spd(y_pred_test, a_utk_test_t).item():.3f}")
    elif c == 1:
        fairs_bma_pp.append(eod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {eod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")
    elif c == 2:
        fairs_bma_pp.append(aod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {aod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")

print("-"*30)
print(f"${np.mean(accs_avg):.3f}_{'{'}\pm {np.std(accs_avg):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_avg):.3f}_{'{'}\pm {np.std(fairs_avg):.3f}{'}'}$")
print(f"${np.mean(accs_bma):.3f}_{'{'}\pm {np.std(accs_bma):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma):.3f}_{'{'}\pm {np.std(fairs_bma):.3f}{'}'}$")
print(f"${np.mean(accs_bma_pp):.3f}_{'{'}\pm {np.std(accs_bma_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma_pp):.3f}_{'{'}\pm {np.std(fairs_bma_pp):.3f}{'}'}$")

------------------------------
$0.822_{\pm 0.006}$ & $0.015_{\pm 0.010}$
$0.843_{\pm 0.002}$ & $0.013_{\pm 0.002}$
$0.859_{\pm 0.003}$ & $0.016_{\pm 0.008}$


Optimize both ensemble and single model for 0.05 constraint

In [250]:
accs_bma_pp, fairs_bma_pp = list(), list()
accs_member_pp, fairs_member_pp = list(), list()

for m in range(len(method_seeds)):

    if verbose: print("-"*20 + f"  seed {m}  " + "-"*20)

    val_m_probits = torch.mean(val_probits[m], dim=0)

    model = DummyPredictor(val_m_probits)

    # Given any trained model that outputs real-valued scores
    fair_clf = RelaxedThresholdOptimizer(
        predictor=lambda X: model(X)[:, -1],   # for sklearn API
        constraint=constraint,
        tolerance=0.05, # fairness constraint tolerance
    )

    # Fit the fairness adjustment on some data
    # This will find the optimal _fair classifier_
    fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

    # overwrite model for predictor
    utk_test_m_probits = torch.mean(utk_test_probits[m], dim=0)
    model.probits = utk_test_m_probits

    # Now you can use `fair_clf` as any other classifier
    # You have to provide group information to compute fair predictions
    y_pred_test = fair_clf(X=torch.tensor(range(len(y_utk_test_t))), group=a_utk_test_t.numpy())
    y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

    if verbose: print("BMA-PP")
    accs_bma_pp.append(accuracy(y_pred_test, y_utk_test_t).item())
    if verbose: print(f"  {(accuracy(y_pred_test, y_utk_test_t).item()):.3f}")
    if c == 0:
        fairs_bma_pp.append(spd(y_pred_test, a_utk_test_t).item())
        if verbose: print(f"  {spd(y_pred_test, a_utk_test_t).item():.3f}")
    elif c == 1:
        fairs_bma_pp.append(eod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {eod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")
    elif c == 2:
        fairs_bma_pp.append(aod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
        if verbose: print(f"  {aod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")

    for mem in range(len(val_probits[m])):
        val_m_probits = val_probits[m][mem]

        model = DummyPredictor(val_m_probits)

        # Given any trained model that outputs real-valued scores
        fair_clf = RelaxedThresholdOptimizer(
            predictor=lambda X: model(X)[:, -1],   # for sklearn API
            constraint=constraint,
            tolerance=0.05, # fairness constraint tolerance
        )

        # Fit the fairness adjustment on some data
        # This will find the optimal _fair classifier_
        fair_clf.fit(X=torch.tensor(range(len(y_val_t))), y=y_val_t.numpy(), group=a_val_t.numpy())

        # overwrite model for predictor
        utk_test_m_probits = utk_test_probits[m][0]
        model.probits = utk_test_m_probits

        # Now you can use `fair_clf` as any other classifier
        # You have to provide group information to compute fair predictions
        y_pred_test = fair_clf(X=torch.tensor(range(len(y_utk_test_t))), group=a_utk_test_t.numpy())
        y_pred_test = torch.tensor(y_pred_test, dtype=torch.long)

        if mem == 0 and verbose : print("Member-PP")
        accs_member_pp.append(accuracy(y_pred_test, y_utk_test_t).item())
        if mem == 0 and verbose : print(f"  {(accuracy(y_pred_test, y_utk_test_t).item()):.3f}")
        if c == 0:
            fairs_member_pp.append(spd(y_pred_test, a_utk_test_t).item())
            if mem == 0 and verbose : print(f"  {spd(y_pred_test, a_utk_test_t).item():.3f}")
        elif c == 1:
            fairs_member_pp.append(eod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
            if mem == 0 and verbose : print(f"  {eod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")
        elif c == 2:
            fairs_member_pp.append(aod(y_pred_test, y_utk_test_t, a_utk_test_t).item())
            if mem == 0 and verbose : print(f"  {aod(y_pred_test, y_utk_test_t, a_utk_test_t).item():.3f}")

print("-"*30)
print(f"${np.mean(accs_bma_pp):.3f}_{'{'}\pm {np.std(accs_bma_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_bma_pp):.3f}_{'{'}\pm {np.std(fairs_bma_pp):.3f}{'}'}$")
print(f"${np.mean(accs_member_pp):.3f}_{'{'}\pm {np.std(accs_member_pp):.3f}{'}'}$", end=" & ")
print(f"${np.mean(fairs_member_pp):.3f}_{'{'}\pm {np.std(fairs_member_pp):.3f}{'}'}$")

------------------------------
$0.859_{\pm 0.002}$ & $0.019_{\pm 0.006}$
$0.816_{\pm 0.014}$ & $0.030_{\pm 0.032}$
